<a href="https://colab.research.google.com/github/swampxx/ML-algorithms-on-ids-datasets/blob/master/ids2018_wednesday_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [7]:
path = "drive/My Drive/security_term_project/"

In [36]:
df = pd.read_csv(path+"ids_2018_wednesday.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613104 entries, 0 to 613103
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Dst Port           613104 non-null  object
 1   Protocol           613104 non-null  object
 2   Timestamp          613104 non-null  object
 3   Flow Duration      613104 non-null  object
 4   Tot Fwd Pkts       613104 non-null  object
 5   Tot Bwd Pkts       613104 non-null  object
 6   TotLen Fwd Pkts    613104 non-null  object
 7   TotLen Bwd Pkts    613104 non-null  object
 8   Fwd Pkt Len Max    613104 non-null  object
 9   Fwd Pkt Len Min    613104 non-null  object
 10  Fwd Pkt Len Mean   613104 non-null  object
 11  Fwd Pkt Len Std    613104 non-null  object
 12  Bwd Pkt Len Max    613104 non-null  object
 13  Bwd Pkt Len Min    613104 non-null  object
 14  Bwd Pkt Len Mean   613104 non-null  object
 15  Bwd Pkt Len Std    613104 non-null  object
 16  Flow Byts/s        6

In [38]:
df['Protocol'].value_counts()

6           215652
6           170064
17          155602
17           61013
0             5578
0             5162
Protocol        33
Name: Protocol, dtype: int64

In [39]:
df["Flow Pkts/s"] = pd.to_numeric(df["Flow Pkts/s"], errors='coerce')
df.loc[df['Protocol'] == "6", 'Protocol'] = 6
df.loc[df['Protocol'] == "17", 'Protocol'] = 17
df.loc[df['Protocol'] == "0", 'Protocol'] = 0
df.dropna(inplace=True)
df.drop('Timestamp', axis=1, inplace=True)
df['Protocol'] = df['Protocol'].astype("category")
one_hot = pd.get_dummies(df["Protocol"], prefix="Protocol")
df = df.drop("Protocol", axis=1)
df = df.join(one_hot)
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df = df.replace("Benign", 0).replace('Infilteration', 1)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606902 entries, 0 to 613103
Data columns (total 81 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           606902 non-null  object 
 1   Flow Duration      606902 non-null  object 
 2   Tot Fwd Pkts       606902 non-null  object 
 3   Tot Bwd Pkts       606902 non-null  object 
 4   TotLen Fwd Pkts    606902 non-null  object 
 5   TotLen Bwd Pkts    606902 non-null  object 
 6   Fwd Pkt Len Max    606902 non-null  object 
 7   Fwd Pkt Len Min    606902 non-null  object 
 8   Fwd Pkt Len Mean   606902 non-null  object 
 9   Fwd Pkt Len Std    606902 non-null  object 
 10  Bwd Pkt Len Max    606902 non-null  object 
 11  Bwd Pkt Len Min    606902 non-null  object 
 12  Bwd Pkt Len Mean   606902 non-null  object 
 13  Bwd Pkt Len Std    606902 non-null  object 
 14  Flow Byts/s        606902 non-null  object 
 15  Flow Pkts/s        606902 non-null  float64
 16  Fl

In [40]:
df.Label.value_counts()

0    538666
1     68236
Name: Label, dtype: int64

In [41]:
all_labels = df.Label.copy().values
all_data = df.drop('Label', axis = 1).values

In [42]:
all_labels.shape, all_data.shape

((606902,), (606902, 80))

### With KBest

In [43]:
featureSelector = SelectKBest(f_classif, k=40)
featureSelector.fit(all_data, all_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [31 33 55 56 57 58 59 60] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SelectKBest(k=40, score_func=<function f_classif at 0x7f794e0f3730>)

In [44]:
df_withoutLabel =  df.drop('Label', axis = 1)
features = df_withoutLabel.columns[featureSelector.get_support()]

In [46]:
features

Index(['Dst Port', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Mean', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Std', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'FIN Flag Cnt', 'SYN Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'CWE Flag Count', 'Pkt Size Avg', 'Fwd Seg Size Avg',
       'Bwd Seg Size Avg', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Idle Mean', 'Idle Max', 'Idle Min', 'Protocol_0', 'Protocol_6',
       'Protocol_17'],
      dtype='object')

### With Feature Importance RandomForestClassifier

In [ ]:
fs = SelectFromModel(RandomForestClassifier())
# learn relationship from training data
fs.fit(all_data, all_labels)

In [ ]:
features3 = df_withoutLabel.columns[fs.get_support()]

In [ ]:
features3

In [49]:
features_all = ['Dst Port', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Mean', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Std', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'FIN Flag Cnt', 'SYN Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'CWE Flag Count', 'Pkt Size Avg', 'Fwd Seg Size Avg',
       'Bwd Seg Size Avg', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Idle Mean', 'Idle Max', 'Idle Min', 'Protocol_0', 'Protocol_6',
       'Protocol_17']

res_feature_list = list(set(features_all))

In [50]:
final_df = df.drop('Label', axis = 1)
final_df = final_df[res_feature_list]
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606902 entries, 0 to 613103
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Protocol_17        606902 non-null  uint8  
 1   Fwd IAT Min        606902 non-null  object 
 2   Init Bwd Win Byts  606902 non-null  object 
 3   Fwd Pkts/s         606902 non-null  object 
 4   Fwd Pkt Len Min    606902 non-null  object 
 5   Pkt Size Avg       606902 non-null  object 
 6   Flow Pkts/s        606902 non-null  float64
 7   Pkt Len Max        606902 non-null  object 
 8   ACK Flag Cnt       606902 non-null  object 
 9   Bwd Pkt Len Min    606902 non-null  object 
 10  Flow IAT Mean      606902 non-null  object 
 11  Bwd Seg Size Avg   606902 non-null  object 
 12  Fwd IAT Mean       606902 non-null  object 
 13  Dst Port           606902 non-null  object 
 14  Pkt Len Min        606902 non-null  object 
 15  PSH Flag Cnt       606902 non-null  object 
 16  Bw

In [53]:
final_df_features_set = final_df.astype(np.float64)
final_df_vals = final_df_features_set.values

In [54]:
np.savetxt(path+"ids2018-wednesday_final_df.txt", final_df_vals)
np.savetxt(path+"ids2018-wednesday_final_df_labels.txt", all_labels)

In [ ]:
q

In [13]:
train_df = df.groupby('Label').apply(pd.DataFrame.sample, frac=0.8).reset_index(level='Label', drop=True)
test_df = df.drop(train_df.index)

In [35]:
df.Label.value_counts()

Benign           540568
Infilteration     68462
Name: Label, dtype: int64

In [14]:
train_df.Label.value_counts()

Benign           432454
Infilteration     54770
Name: Label, dtype: int64

In [15]:
train_df.count()

Dst Port         487224
Protocol         487224
Flow Duration    487224
Tot Fwd Pkts     487224
Tot Bwd Pkts     487224
                  ...  
Idle Mean        487224
Idle Std         487224
Idle Max         487224
Idle Min         487224
Label            487224
Length: 79, dtype: int64

In [16]:
test_df.count()

Dst Port         121806
Protocol         121806
Flow Duration    121806
Tot Fwd Pkts     121806
Tot Bwd Pkts     121806
                  ...  
Idle Mean        121806
Idle Std         121806
Idle Max         121806
Idle Min         121806
Label            121806
Length: 79, dtype: int64

In [17]:
def process_df(dataFrame):
    df1 = dataFrame.replace('Benign', 0).replace('Infilteration', 1)
    df1['Protocol'] = df1['Protocol'].astype("category")
    one_hot = pd.get_dummies(df1["Protocol"], prefix="Protocol")
    df1 = df1.drop("Protocol", axis=1)
    df1 = df1.join(one_hot)
    res = df1[~df1.isin([np.nan, np.inf, -np.inf]).any(1)]

    return res

In [18]:
train_set = process_df(train_df)
train_labels = train_set.Label.copy()
train_set.drop('Label', axis=1, inplace=True)


test_set = process_df(test_df)
test_labels = test_set.Label.copy()
test_set.drop('Label', axis=1, inplace=True)

In [19]:
train_set = train_set.astype(np.float64)
test_set = test_set.astype(np.float64)

In [20]:
X, y = train_set.values, train_labels.values
X_test, y_test = test_set.values, test_labels.values

In [28]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 485541 entries, 356606 to 386817
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           485541 non-null  float64
 1   Flow Duration      485541 non-null  float64
 2   Tot Fwd Pkts       485541 non-null  float64
 3   Tot Bwd Pkts       485541 non-null  float64
 4   TotLen Fwd Pkts    485541 non-null  float64
 5   TotLen Bwd Pkts    485541 non-null  float64
 6   Fwd Pkt Len Max    485541 non-null  float64
 7   Fwd Pkt Len Min    485541 non-null  float64
 8   Fwd Pkt Len Mean   485541 non-null  float64
 9   Fwd Pkt Len Std    485541 non-null  float64
 10  Bwd Pkt Len Max    485541 non-null  float64
 11  Bwd Pkt Len Min    485541 non-null  float64
 12  Bwd Pkt Len Mean   485541 non-null  float64
 13  Bwd Pkt Len Std    485541 non-null  float64
 14  Flow Byts/s        485541 non-null  float64
 15  Flow Pkts/s        485541 non-null  float64
 1

In [29]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121361 entries, 5 to 613101
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           121361 non-null  float64
 1   Flow Duration      121361 non-null  float64
 2   Tot Fwd Pkts       121361 non-null  float64
 3   Tot Bwd Pkts       121361 non-null  float64
 4   TotLen Fwd Pkts    121361 non-null  float64
 5   TotLen Bwd Pkts    121361 non-null  float64
 6   Fwd Pkt Len Max    121361 non-null  float64
 7   Fwd Pkt Len Min    121361 non-null  float64
 8   Fwd Pkt Len Mean   121361 non-null  float64
 9   Fwd Pkt Len Std    121361 non-null  float64
 10  Bwd Pkt Len Max    121361 non-null  float64
 11  Bwd Pkt Len Min    121361 non-null  float64
 12  Bwd Pkt Len Mean   121361 non-null  float64
 13  Bwd Pkt Len Std    121361 non-null  float64
 14  Flow Byts/s        121361 non-null  float64
 15  Flow Pkts/s        121361 non-null  float64
 16  Fl

In [30]:
train_labels.value_counts()

0    430955
1     54586
Name: Label, dtype: int64

In [31]:
test_labels.value_counts()

0    107711
1     13650
Name: Label, dtype: int64

## Feature Selection


### With SelectKBest and f_classif

In [21]:
featureSelector = SelectKBest(f_classif, k=30)
X_new = featureSelector.fit_transform(X, y)
features = train_set.columns[featureSelector.get_support(indices=True)]

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [31 33 55 56 57 58 59 60] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [22]:
print(features)

Index(['Dst Port', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Std', 'Bwd Pkt Len Min', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Std', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'FIN Flag Cnt', 'SYN Flag Cnt', 'PSH Flag Cnt',
       'CWE Flag Count', 'Fwd Seg Size Avg', 'Init Fwd Win Byts',
       'Init Bwd Win Byts', 'Protocol_0', 'Protocol_17'],
      dtype='object')


In [23]:
X_test_kbest, y_test = test_set[features].values, test_labels.values

In [ ]:
#np.savetxt(path+"train_x_processed_kbest.txt", X_new)
#np.savetxt(path+"train_y_processed_kbest.txt", y)

#np.savetxt(path+"test_x_processed_kbest.txt", X_test_kbest)
#np.savetxt(path+"test_y_processed_kbest.txt", y_test)

### Feature Selection with Importance

In [24]:
X.shape

(485541, 80)

In [25]:
fs = SelectFromModel(RandomForestClassifier())
# learn relationship from training data
fs.fit(X, y)
# transform train input data
X_train_fs = fs.transform(X)
# transform test input data
X_test_fs = fs.transform(X_test)

In [26]:
X_train_fs.shape

(485541, 21)

In [27]:
features = train_set.columns[fs.get_support()]
print(features)

Index(['Dst Port', 'Flow Duration', 'Flow Byts/s', 'Flow Pkts/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Init Fwd Win Byts'],
      dtype='object')


In [ ]:
#np.savetxt(path+"train_x_processed_fi.txt", X_train_fs)
#np.savetxt(path+"train_y_processed_fi.txt", y)

#np.savetxt(path+"test_x_processed_fi.txt", X_test_fs)
#np.savetxt(path+"test_y_processed_fi.txt", y_test)